# Data Science and Big Data Analytics Mini Project
# Movie Recommendation System
## Group members:
## 1. Sanket Nagare
## 2. Yukta Bhatkar
## 3. Rupal agrawal
## 4. Vaishnavi Dalvi


In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
#Compute cosine similarity between samples in X and Y.
from sklearn.feature_extraction.text import CountVectorizer
#CountVectorizer It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.

### Using dataset given in sppu syllabus

In [12]:
url = 'https://raw.githubusercontent.com/rashida048/Some-NLP-Projects/master/movie_dataset.csv'

In [13]:
df = pd.read_csv(url)

In [17]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


### Choose the features to be used for the model. We do not need to use all the features.

In [16]:
features = ['keywords','cast','genres','director']

### Combine those features and make one column out of those four columns.

In [18]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

### If we have null values, that may create problems later on in the algorithm. Fill the null values with empty strings.

In [19]:
for feature in features:
    df[feature] = df[feature].fillna('')
    
df["combined_features"] = df.apply(combine_features,axis=1)

### Fit and transform the data into the ‘count vectorizer’ function that prepares the data for the vector representation. When you pass the text data through the ‘count vectorizer’ function, it returns a matrix of the number count of each word.

In [20]:
#Create count matrix from this new combined column
cv = CountVectorizer() 
count_matrix = cv.fit_transform(df["combined_features"])

### Use ‘cosine_similarity’ to find the similarity. This is a dynamic way of finding the similarity that measures the cosine angle between two vectors in a multi-dimensional space. In this way, the size of the documents does not matter. The documents could be far apart by the Euclidean distance but their cosine angle can be similar.

In [21]:
#Compute the Cosine Similarity based on the count_matrix
cosine_sim = cosine_similarity(count_matrix)
#This ‘cosine_sim’ is a two-dimensional matrix

### We need to define two functions. One of the functions returns the title from the index and the other one returns the index from the title.

In [22]:
def find_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def find_index_from_title(title):
    return df[df.title == title]["index"].values[0]

### Take a movie that our user likes. Let’s take ‘Star Wars’. Then find the index of this movie using the function above.

In [23]:
movie = "Star Wars"
movie_index = find_index_from_title(movie)

### The index of ‘Star Wars’ is 2912. As I mentioned earlier that ‘cosine_sim’ in step 6 is a matrix of the similarity coefficients. Row 2912 of that matrix should provide the similarity coefficients of all the movies with ‘Star Wars’. So, find row 2912 of the matrix ‘cosine_sim’.

In [24]:
similar_movies = list(enumerate(cosine_sim[movie_index]))
# we are using enumerate to get the index and the coefficients. ‘similar_movies’ is a list of tuples that contains index and coefficients.

### Sort the list ‘similar_movies’ by the coefficients in the reverse order. That way, the highest coefficients will be on top.
### We are not taking the first one from the list because the top one in the list will be ‘Star Wars’ itself.

In [25]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

### Use the function ‘find_title_from_index’ to get the top five similar movies to the ‘Star Wars’.

In [27]:
i=0
for element in sorted_similar_movies:
    print(find_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

The Empire Strikes Back
Return of the Jedi
Star Wars: Episode II - Attack of the Clones
Star Wars: Episode III - Revenge of the Sith
Star Wars: Episode I - The Phantom Menace
The Helix... Loaded
